## Final proposal: Opioid prescription
#### Shihui Wang sw2804@nyu.edu
This project will investigate how opioid prescriptions are related to the practioners' field, geography and local income in the US.
The key dataset is a Socrata API. It contains information about prescribers name, practice address and type, as well as opioid prescribing rate. I combined it with census api on income per capita in 2016 by zipcode. 
I anticipate the project to have the following parts:
- Basic statistics and graphs on prescription by professions, state income and opioid death rate by state.
- Visualization of prescription on overdose death and local per capita income.
- A national map of opioid prescription and opioid overdose death by state.

In [3]:
import pandas as pd
from sodapy import Socrata
import numpy  as np
import matplotlib.pyplot as plt
from census import Census
from us import states
import requests
import json

In [54]:
client = Socrata("data.cms.gov", 'jwf3Ur2ya3N6XrQBYjgV4vdXw',username="sw2804@nyu.edu",
                 password="mW@ng4*0o!")

In [55]:
results = client.get("3hwr-y534", limit=1072997)

In [56]:
results_df = pd.DataFrame.from_records(results)

In [57]:
results_df.head()

,npi,nppes_provider_first_name,nppes_provider_last_org_name,nppes_provider_state,nppes_provider_zip5,opioid_claim_count,opioid_rate,specialty_desc,total_claim_count
0,1831127620,FRED,BRESSLER,TX,77030,NaN,NaN,Otolaryngology,134
1,1588095855,REID,BREWSTER,CA,93550,NaN,NaN,Dentist,22
2,1144293044,BRUCE,BRIDEWELL,FL,34135,NaN,NaN,Family Practice,698
3,1255428645,THOMAS,BRIDGMAN,WA,98101,NaN,NaN,Dentist,40
4,1619082526,MARTIN,BRIGGS,LA,70525,NaN,NaN,Dentist,33


In [58]:
opioid = results_df.dropna()
opioid.head()

,npi,nppes_provider_first_name,nppes_provider_last_org_name,nppes_provider_state,nppes_provider_zip5,opioid_claim_count,opioid_rate,specialty_desc,total_claim_count
7,1326336751,LOUIS,BRITT,TN,38018,0,0,General Surgery,30
14,1841342003,CHARLES,BROWN,CA,92618,0,0,Dentist,16
15,1811325749,KARA,BROWN,OR,97015,0,0,Pharmacist,11
26,1194710434,JILL,BUCK,IN,46143,0,0,Optometry,81
31,1417264565,RYAN,BURKE,NM,87106,0,0,Pharmacist,11


In [59]:
opioid.rename(columns={'nppes_provider_zip5':'zipcode'}, inplace=True)
opioid

/Applications/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:2746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  **kwargs)


,npi,nppes_provider_first_name,nppes_provider_last_org_name,nppes_provider_state,zipcode,opioid_claim_count,opioid_rate,specialty_desc,total_claim_count
7,1326336751,LOUIS,BRITT,TN,38018,0,0,General Surgery,30
14,1841342003,CHARLES,BROWN,CA,92618,0,0,Dentist,16
15,1811325749,KARA,BROWN,OR,97015,0,0,Pharmacist,11
26,1194710434,JILL,BUCK,IN,46143,0,0,Optometry,81
31,1417264565,RYAN,BURKE,NM,87106,0,0,Pharmacist,11
34,1235186511,MARY,BURTON,UT,84105,0,0,Pediatric Medicine,151
54,1265506067,LARISA,CANTER,CA,92660,0,0,Ophthalmology,743
60,1164418778,BRENDA,CARRELL,MO,65804,0,0,Optometry,75
68,1649243320,RAYMOND,CAST,FL,32209,0,0,Physician Assistant,60
72,1639395957,KIMBERLY,CERTA,VA,22192,0,0,Dentist,12


Now getting income per capita at state level and zipcode level in 2016 data from census

In [82]:
my_api_key = '34caee0abc641aead3785ea57203c35b70def3ee'
c = Census(my_api_key)
code = ("NAME","B19301_001E")    #PER CAPITA INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS)
state_wealth = c.acs5.get(code, {'for': 'state:* '})
state_wealth = pd.DataFrame(state_wealth)
state_wealth.rename(columns={'NAME':'State','B19301_001E':'state_per_capita_income'}, inplace=True)
state_wealth.drop('state',axis=1,inplace=True)
state_wealth

,state_per_capita_income,State
0,24736.0,Alabama
1,34191.0,Alaska
2,26686.0,Arizona
3,23401.0,Arkansas
4,31458.0,California
5,33230.0,Colorado
6,39906.0,Connecticut
7,31118.0,Delaware
8,48781.0,District of Columbia
9,27598.0,Florida


In [61]:
zipcode = pd.DataFrame(c.acs5.get(code, 
                                         {'for': 'zip code tabulation area: *'}))

zipcode.head()

,B19301_001E,NAME,zip code tabulation area
0,30430.0,ZCTA5 01001,01001
1,26072.0,ZCTA5 01002,01002
2,3829.0,ZCTA5 01003,01003
3,32169.0,ZCTA5 01005,01005
4,36359.0,ZCTA5 01007,01007


In [62]:
zipcode.rename(columns={'zip code tabulation area':'zipcode','B19301_001E':'per_capita_income'}, inplace=True)

Merging these two datasets together by zipcode

In [66]:
combo = pd.merge(opioid, zipcode,# left df, right df
                 on='zipcode',       # link with cntry
                 how='left',        # add to left...does this matter here? inner: inner section of the keys, 
                                     # outter: takes all the keys on both dataframe
                 indicator=True)  # Tells us what happend

combo.rename(columns={'nppes_provider_state':'Abbreviation','nppes_provider_first_name':'first_name','nppes_provider_last_org_name':'last_name'}, inplace=True)
combo

,npi,first_name,last_name,Abbreviation,zipcode,opioid_claim_count,opioid_rate,specialty_desc,total_claim_count,per_capita_income,NAME,_merge
0,1326336751,LOUIS,BRITT,TN,38018,0,0,General Surgery,30,32143.0,ZCTA5 38018,both
1,1841342003,CHARLES,BROWN,CA,92618,0,0,Dentist,16,49991.0,ZCTA5 92618,both
2,1811325749,KARA,BROWN,OR,97015,0,0,Pharmacist,11,29906.0,ZCTA5 97015,both
3,1194710434,JILL,BUCK,IN,46143,0,0,Optometry,81,30999.0,ZCTA5 46143,both
4,1417264565,RYAN,BURKE,NM,87106,0,0,Pharmacist,11,27810.0,ZCTA5 87106,both
5,1235186511,MARY,BURTON,UT,84105,0,0,Pediatric Medicine,151,39569.0,ZCTA5 84105,both
6,1265506067,LARISA,CANTER,CA,92660,0,0,Ophthalmology,743,78374.0,ZCTA5 92660,both
7,1164418778,BRENDA,CARRELL,MO,65804,0,0,Optometry,75,31186.0,ZCTA5 65804,both
8,1649243320,RAYMOND,CAST,FL,32209,0,0,Physician Assistant,60,13113.0,ZCTA5 32209,both
9,1639395957,KIMBERLY,CERTA,VA,22192,0,0,Dentist,12,40052.0,ZCTA5 22192,both


In [4]:
state=pd.read_csv('states.csv')

In [68]:
combo2=pd.merge(combo,state,on='Abbreviation',how='left')
combo2

,npi,first_name,last_name,Abbreviation,zipcode,opioid_claim_count,opioid_rate,specialty_desc,total_claim_count,per_capita_income,NAME,_merge,State
0,1326336751,LOUIS,BRITT,TN,38018,0,0,General Surgery,30,32143.0,ZCTA5 38018,both,Tennessee
1,1841342003,CHARLES,BROWN,CA,92618,0,0,Dentist,16,49991.0,ZCTA5 92618,both,California
2,1811325749,KARA,BROWN,OR,97015,0,0,Pharmacist,11,29906.0,ZCTA5 97015,both,Oregon
3,1194710434,JILL,BUCK,IN,46143,0,0,Optometry,81,30999.0,ZCTA5 46143,both,Indiana
4,1417264565,RYAN,BURKE,NM,87106,0,0,Pharmacist,11,27810.0,ZCTA5 87106,both,New Mexico
5,1235186511,MARY,BURTON,UT,84105,0,0,Pediatric Medicine,151,39569.0,ZCTA5 84105,both,Utah
6,1265506067,LARISA,CANTER,CA,92660,0,0,Ophthalmology,743,78374.0,ZCTA5 92660,both,California
7,1164418778,BRENDA,CARRELL,MO,65804,0,0,Optometry,75,31186.0,ZCTA5 65804,both,Missouri
8,1649243320,RAYMOND,CAST,FL,32209,0,0,Physician Assistant,60,13113.0,ZCTA5 32209,both,Florida
9,1639395957,KIMBERLY,CERTA,VA,22192,0,0,Dentist,12,40052.0,ZCTA5 22192,both,Virginia


I also want to get death by opioid overdose data

In [70]:
url='https://data.opendatasoft.com/api/records/1.0/search/?dataset=prescription-opioid-overdose-deaths-and-death-rate-per-100000-population-age-adj%40public&rows=867&facet=prescription_opioid_overdose_deaths_missing_reason&facet=footnotes&facet=location&facet=year'

In [71]:
r=requests.get(url)

In [72]:
print(r.headers["content-type"])


application/json; charset=utf-8


In [73]:
data= json.loads(r.content.decode("utf-8"))

In [74]:
df=pd.DataFrame(data['records'])
dff=pd.DataFrame(df['fields'].tolist())
dff.drop('prescription_opioid_overdose_deaths_missing_reason',axis=1,inplace=True)
death=dff.set_index('year').loc['2015']
death.rename(columns={'location':'State'}, inplace=True)
death

,State,prescription_opioid_overdose_deaths
year,,
2015,Maryland,747.0
2015,Minnesota,221.0
2015,West Virginia,512.0
2015,Washington,400.0
2015,Montana,40.0
2015,Idaho,73.0
2015,Alabama,174.0
2015,Arkansas,186.0
2015,Nebraska,48.0


In [77]:
combo3=pd.merge(combo2,death,on='State',how='left')
combo3.drop(['NAME','_merge'],axis=1,inplace=True)
combo3

,npi,first_name,last_name,Abbreviation,zipcode,opioid_claim_count,opioid_rate,specialty_desc,total_claim_count,per_capita_income,State,prescription_opioid_overdose_deaths
0,1326336751,LOUIS,BRITT,TN,38018,0,0,General Surgery,30,32143.0,Tennessee,853.0
1,1841342003,CHARLES,BROWN,CA,92618,0,0,Dentist,16,49991.0,California,1414.0
2,1811325749,KARA,BROWN,OR,97015,0,0,Pharmacist,11,29906.0,Oregon,228.0
3,1194710434,JILL,BUCK,IN,46143,0,0,Optometry,81,30999.0,Indiana,279.0
4,1417264565,RYAN,BURKE,NM,87106,0,0,Pharmacist,11,27810.0,New Mexico,211.0
5,1235186511,MARY,BURTON,UT,84105,0,0,Pediatric Medicine,151,39569.0,Utah,423.0
6,1265506067,LARISA,CANTER,CA,92660,0,0,Ophthalmology,743,78374.0,California,1414.0
7,1164418778,BRENDA,CARRELL,MO,65804,0,0,Optometry,75,31186.0,Missouri,439.0
8,1649243320,RAYMOND,CAST,FL,32209,0,0,Physician Assistant,60,13113.0,Florida,1417.0
9,1639395957,KIMBERLY,CERTA,VA,22192,0,0,Dentist,12,40052.0,Virginia,548.0


In [83]:
final_df=pd.merge(combo3,state_wealth,on='State',how='left')
final_df

,npi,first_name,last_name,Abbreviation,zipcode,opioid_claim_count,opioid_rate,specialty_desc,total_claim_count,per_capita_income,State,prescription_opioid_overdose_deaths,state_per_capita_income
0,1326336751,LOUIS,BRITT,TN,38018,0,0,General Surgery,30,32143.0,Tennessee,853.0,26019.0
1,1841342003,CHARLES,BROWN,CA,92618,0,0,Dentist,16,49991.0,California,1414.0,31458.0
2,1811325749,KARA,BROWN,OR,97015,0,0,Pharmacist,11,29906.0,Oregon,228.0,28822.0
3,1194710434,JILL,BUCK,IN,46143,0,0,Optometry,81,30999.0,Indiana,279.0,26117.0
4,1417264565,RYAN,BURKE,NM,87106,0,0,Pharmacist,11,27810.0,New Mexico,211.0,24459.0
5,1235186511,MARY,BURTON,UT,84105,0,0,Pediatric Medicine,151,39569.0,Utah,423.0,25600.0
6,1265506067,LARISA,CANTER,CA,92660,0,0,Ophthalmology,743,78374.0,California,1414.0,31458.0
7,1164418778,BRENDA,CARRELL,MO,65804,0,0,Optometry,75,31186.0,Missouri,439.0,27044.0
8,1649243320,RAYMOND,CAST,FL,32209,0,0,Physician Assistant,60,13113.0,Florida,1417.0,27598.0
9,1639395957,KIMBERLY,CERTA,VA,22192,0,0,Dentist,12,40052.0,Virginia,548.0,34967.0
